# 지표 정리

#### 기본 세팅

In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

### 데이터 

In [2]:
glob.glob("data_crawler/*")

['data_crawler/cleaning',
 'data_crawler/kor_index_code_fundamental',
 'data_crawler/index_code_master',
 'data_crawler/kor_market_cap',
 'data_crawler/ohlcv_df_raw.csv',
 'data_crawler/sample.csv',
 'data_crawler/snp500_daily',
 'data_crawler/bitcoin',
 'data_crawler/usd_krw',
 'data_crawler/kor_stock_ohlcv',
 'data_crawler/kor_ticker_list',
 'data_crawler/kor_index_list_df',
 'data_crawler/kor_index_ohlcv',
 'data_crawler/cron_test_kor_ticker_list',
 'data_crawler/kor_stock_fundamental',
 'data_crawler/snp500_ticker_list',
 'data_crawler/df_raw_total.csv']

In [3]:
glob.glob("data_crawler/cleaning/*")

['data_crawler/cleaning/bitcoin',
 'data_crawler/cleaning/kor_stock_ohlcv',
 'data_crawler/cleaning/kor_index_ohlcv']

In [4]:
glob.glob("data_crawler/cleaning/kor_stock_ohlcv/*")

['data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kospi.parquet']

In [5]:
glob.glob("data_crawler/cleaning/kor_index_ohlcv/*")

['data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet',
 'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet']

In [10]:
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet')
kor_stock_ohlcv_kospi = t.to_pandas()

t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet')
kor_stock_ohlcv_anal_kospi = t.to_pandas()

In [11]:
kor_stock_ohlcv_kospi.head(3)

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,MA5,MA20,MA60,MA120,upper,lower,MACD_DIFF,MACD,MACD_Signal,RSI
0,2023-01-09 00:00:00+00:00,59700,60700,59600,60700.0,18640107,2.881356,005930,삼성전자,KOSPI,58220.0,58315.0,59373.333333,58841.666667,61407.749586,55222.250414,297.792250,-382.242616,-680.034867,59.579300
1,2023-01-10 00:00:00+00:00,60200,61100,59900,60400.0,14859797,-0.494234,005930,삼성전자,KOSPI,59220.0,58360.0,59436.666667,58829.166667,61545.215848,55174.784152,402.537204,-176.863361,-579.400566,58.001471
2,2023-01-11 00:00:00+00:00,61000,61200,60300,60500.0,12310751,0.165563,005930,삼성전자,KOSPI,59760.0,58400.0,59503.333333,58825.833333,61670.473972,55129.526028,458.751698,-5.960943,-464.712641,58.396977


In [12]:
kor_stock_ohlcv_anal_kospi.head(3)

,date,ticker,corp_name,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-01-09 00:00:00+00:00,005930,삼성전자,KOSPI,60700,-,-,-,-,-,-
1,2023-01-10 00:00:00+00:00,005930,삼성전자,KOSPI,60400,골든크로스(매수),-,-,-,-,-
2,2023-01-11 00:00:00+00:00,005930,삼성전자,KOSPI,60500,-,-,-,-,-,-
